# 모두투어 여행 패키지 크롤링

Selenium과 BeautifulSoup을 사용하여 모드투어 웹사이트에서 여행 패키지 정보를 크롤링합니다. 크롤링된 데이터는 제목, 가격, 출발일, 여행 기간, 목적지로 구성되어 있으며, 해당 정보를 추출하여 출력합니다.

## 0. 사용자 입력 받기

In [49]:
# 일본 지역 코드 딕셔너리
japan_areas = {
    "동경": "61852414-60F0-43BE-7309-08DB9ED3CDC3",
    "오사카": "A934E79F-B8CC-4A1A-730E-08DB9ED3CDC3",
    "큐슈": "55DE74C1-31F9-4770-730F-08DB9ED3CDC3",
    "북해도": "B3005E50-5FE5-4602-7310-08DB9ED3CDC3",
    "알펜루트": "D5AB8052-AC82-4D47-C820-08DBA46004A4",
    "오키나와": "9A91BD36-A153-473D-C821-08DBA46004A4",
    "시코쿠/주고쿠": "43C600DE-B953-4130-7311-08DB9ED3CDC3",
    "도호쿠": "5DD94C16-0E5C-40CD-7312-08DB9ED3CDC3",
    "일본(선박)": "5DD94C16-0E5C-40CD-7312-08DB9ED3CDC3",
    "일본 성지순례": "77F96104-B838-4655-DD0D-08DBF0735E30"
}

# 출발지 코드 딕셔너리
starting_points = {
    "서울": "SEL",
    "김포": "GMP",
    "부산": "PUS",
    "대구": "TAE",
    "제주": "CJU",
    "청주": "CJJ"
}

# 사용자에게 지역 선택을 요청
print("다음 중에서 여행하고 싶은 일본 지역을 선택하세요:")
for key in japan_areas.keys():
    print(f"- {key}")

# 사용자로부터 지역 입력받기
selected_area = input("지역을 입력하세요: ")

# 선택된 지역의 코드 가져오기
if selected_area in japan_areas:
    area_id = japan_areas[selected_area]
else:
    print("잘못된 입력입니다. 프로그램을 종료합니다.")
    exit()

print(f"{selected_area}을(를) 선택하셨습니다.")

# 사용자에게 출발지 선택을 요청
print("다음 중에서 출발지를 선택하세요:")
for key in starting_points.keys():
    print(f"- {key}")

# 사용자로부터 출발지 입력받기
selected_starting_point = input("출발지를 입력하세요: ")

# 선택된 출발지의 코드 가져오기
if selected_starting_point in starting_points:
    starting_point_code = starting_points[selected_starting_point]
else:
    print("잘못된 입력입니다. 프로그램을 종료합니다.")
    exit()

print(f"{selected_starting_point}에서 출발하는 여행을 선택하셨습니다.")
print("-----------------------------------------------------")
print(f"{selected_area}에서 {selected_starting_point}로 가능 패키지를 검색합니다.")

다음 중에서 여행하고 싶은 일본 지역을 선택하세요:
- 동경
- 오사카
- 큐슈
- 북해도
- 알펜루트
- 오키나와
- 시코쿠/주고쿠
- 도호쿠
- 일본(선박)
- 일본 성지순례
지역을 입력하세요: 동경
동경을(를) 선택하셨습니다.
다음 중에서 출발지를 선택하세요:
- 서울
- 김포
- 부산
- 대구
- 제주
- 청주
출발지를 입력하세요: 서울
서울에서 출발하는 여행을 선택하셨습니다.
-----------------------------------------------------
동경에서 서울로 가능 패키지를 검색합니다.


## 1. 필요한 패키지 설치

In [50]:
# 크롤링 라이브러리 설치
!pip install webdriver-manager
!pip install selenium

# 필요한 패키지 및 Chrome 설치
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install

# ChromeDriver 다운로드 및 설치
!wget -O /tmp/chromedriver.zip https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!yes|unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not

## 2. 필요한 모듈 임포트

In [51]:
# 2. 필요한 모듈 임포트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup


## 3. Chrome 옵션 설정

In [52]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Headless 모드 설정 (브라우저 UI 없이 실행)
chrome_options.add_argument("--no-sandbox")  # 샌드박스 모드 비활성화
chrome_options.add_argument("--disable-dev-shm-usage")  # /dev/shm 사용 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 비활성화
chrome_options.add_argument("--remote-debugging-port=9222")  # 원격 디버깅 포트 설정
chrome_options.add_argument("--window-size=1920x1080")  # 윈도우 크기 설정
chrome_options.add_argument('--disable-extensions')  # 브라우저 확장 기능 비활성화
chrome_options.add_argument('--headless')  # Headless 모드 중복 설정
chrome_options.add_argument('--disable-dev-shm-usage')  # /dev/shm 사용 비활성화 중복 설정
chrome_options.add_argument('--no-sandbox')  # 샌드박스 모드 비활성화 중복 설정


## 4. Chrome 드라이버 초기화

In [53]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

## 5. 크롤링할 URL 설정

In [54]:
# 선택된 지역 및 출발지에 따른 URL 생성
url = f"https://www.modetour.com/package/search-result?query={{\"type\":\"overseas\",\"searchFrom\":\"2024-08-20\",\"searchTo\":\"2025-07-20\",\"areaId\":\"{area_id}\",\"filter\":{{\"flight\":{{}},\"startingPoint\":[\"{starting_point_code}\"],\"tourCondition\":{{}}}}}}"

## 6. URL 열기 및 페이지 로드 대기

In [55]:
driver.get(url)

# 페이지가 로드될 때까지 대기
time.sleep(5)

## 7. 페이지 끝까지 스크롤하여 모든 데이터 로드

In [56]:
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


## 8. 페이지 소스를 파싱하고 여행 패키지 정보 추출

In [57]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
packages = soup.find_all('div', class_='first:mt-[0px] mt-[30px]')

## 9. 필요한 정보 추출 및 출력

In [58]:

for package in packages:
    # 패키지 제목 추출
    title = package.find('h3', class_='font-semibold mb-3 text-[20px] h-6 leading-[26px] line-clamp-1')
    title_text = title.text.strip() if title else 'N/A'

    # 패키지 가격 추출
    price = package.find('div', class_='font-bold text-[24px] h-[30px] leading-6')
    price_text = price.text.strip() if price else 'N/A'

    # 출발일 추출
    departure = package.find('span', class_='text-[14px] font-normal text-[#555555] leading-[17px]')
    departure_text = departure.text.strip() if departure else 'N/A'

    # 여행 기간 추출
    duration_div = package.find('div', class_='text-[14px] font-normal text-[#555555] leading-[17px]')
    duration_text = duration_div.text.strip() if duration_div else 'N/A'

    # 여행 목적지 추출
    destination_div = package.find_all('div', class_='text-[14px] font-normal text-[#555555] leading-[17px]')[1]
    destination_text = destination_div.text.strip() if destination_div else 'N/A'

    # 추출된 정보 출력
    print(f"Title: {title_text}")
    print(f"Price: {price_text}")
    print(f"Departure: {departure_text}")
    print(f"Duration: {duration_text}")
    print(f"Destination: {destination_text}")
    print('-' * 40)


Title: 도쿄 패키지 3일(전일)
Price: 629,900원 ~
Departure: 출발 · 서울
Duration: 2박3일 / 3박4일
Destination: 도쿄, 요코하마, 닛꼬, 하코네, 시즈오카
----------------------------------------
Title: 도쿄 패키지 3일(1일자유)
Price: 699,900원 ~
Departure: 출발 · 서울
Duration: 2박3일
Destination: 도쿄, 요코하마
----------------------------------------
Title: 도쿄 패키지 4일(전일)
Price: 1,049,900원 ~
Departure: 출발 · 서울
Duration: 3박4일
Destination: 도쿄, 요코하마, 닛꼬, 하코네, 시즈오카
----------------------------------------
Title: 도쿄 패키지 4일(1일자유)
Price: 729,900원 ~
Departure: 출발 · 서울
Duration: 3박4일
Destination: 도쿄, 요코하마
----------------------------------------
Title: 시즈오카 패키지 4일(도쿄연계)
Price: 749,900원 ~
Departure: 출발 · 서울
Duration: 3박4일
Destination: 시즈오카, 도쿄, 하코네
----------------------------------------
Title: [온천호텔 숙박+다색골프]동경 나리타  골프 4일
Price: 1,659,900원 ~
Departure: 출발 · 서울
Duration: 3박4일
Destination: 도쿄
----------------------------------------
Title: [골프텔/2인가능] 일본 도쿄  골프 3일[실속/특급]
Price: 1,189,900원 ~
Departure: 출발 · 서울
Duration: 2박3일
Destination: 나리타, 도쿄
---------

# 10. 드라이버 종료


In [59]:
driver.quit()

## 11. 상세페이지

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.binary_location = "/usr/bin/google-chrome"
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument('--disable-extensions')

# Chrome 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# URL 열기
url = "https://www.modetour.com/package/search-result?query={\"type\":\"overseas\",\"searchFrom\":\"2024-08-20\",\"searchTo\":\"2025-07-20\",\"areaId\":\"61852414-60F0-43BE-7309-08DB9ED3CDC3\",\"filter\":{\"flight\":{},\"startingPoint\":[\"SEL\"],\"tourCondition\":{}}}"
driver.get(url)

# 페이지가 로드될 때까지 대기
time.sleep(5)

# 첫 번째 패키지 클릭
first_package = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-hidden="true"].cursor-pointer'))
)
first_package.click()

# 페이지가 로드될 때까지 대기
time.sleep(2)

# customScroll 클릭 (상세 보기를 열기 위해)
custom_scroll = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.customScroll'))
)
custom_scroll.click()

# 추가적인 정보 로드 대기 및 처리
time.sleep(5)

# 필요한 정보를 추출하거나 추가 동작을 수행
package_details_html = driver.page_source

# 여기에서 BeautifulSoup을 사용해 필요한 데이터를 추출할 수 있음
# soup = BeautifulSoup(package_details_html, 'html.parser')
# 필요한 데이터 추출

print("패키지 상세 정보가 로드되었습니다.")

# 드라이버 종료
driver.quit()


패키지 상세 정보가 로드되었습니다.


In [74]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.binary_location = "/usr/bin/google-chrome"
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument('--disable-extensions')

# Chrome 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# URL 열기
url = "https://www.modetour.com/package/search-result?query={\"type\":\"overseas\",\"searchFrom\":\"2024-08-20\",\"searchTo\":\"2025-07-20\",\"areaId\":\"61852414-60F0-43BE-7309-08DB9ED3CDC3\",\"filter\":{\"flight\":{},\"startingPoint\":[\"SEL\"],\"tourCondition\":{}}}"
driver.get(url)

# 페이지가 로드될 때까지 대기
time.sleep(5)

# 첫 번째 패키지 클릭
first_package = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-hidden="true"].cursor-pointer'))
)
first_package.click()

# 페이지가 로드될 때까지 대기
time.sleep(2)

# customScroll 클릭 (상세 보기를 열기 위해)
custom_scroll = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.customScroll'))
)
custom_scroll.click()

# 추가적인 정보 로드 대기 및 처리
time.sleep(5)

# 필요한 정보를 추출하거나 추가 동작을 수행
package_details_html = driver.page_source

# BeautifulSoup 객체 생성
soup = BeautifulSoup(package_details_html, 'html.parser')

# 호텔 정보를 담을 리스트
hotel_info_list = []

# 호텔 정보 테이블에서 각 열을 추출
table_columns = soup.find_all('div', class_='table_column ')
print(f"Found {len(table_columns)} columns")  # 확인을 위한 출력

for column in table_columns:
    day = column.find('div', class_='header_column').text.strip()
    date = column.find_all('div', class_='td')[1].text.strip()
    city = column.find_all('div', class_='td')[2].text.strip()

    hotels = column.find_all('div', class_='td underline px-[20px] line-clamp-1 !justify-normal !flex-col !items-start space-y-2')
    hotel_names = [hotel.get_text(" ", strip=True) for hotel in hotels]

    # 정보를 딕셔너리로 저장
    hotel_info = {
        "day": day,
        "date": date,
        "city": city,
        "hotels": hotel_names
    }

    hotel_info_list.append(hotel_info)
    print(hotel_info)  # 각 단계에서 추출된 정보 출력

# 추출한 호텔 정보 출력
for info in hotel_info_list:
    print(f"일차: {info['day']}")
    print(f"날짜: {info['date']}")
    print(f"도시: {info['city']}")
    print("호텔 목록:")
    for hotel in info['hotels']:
        print(f"- {hotel}")
    print("\n")


# 드라이버 종료
driver.quit()


Found 0 columns


In [71]:
print(package_details_html)  # 페이지 소스 출력


<html lang="ko"><head><style data-rc-order="prepend" rc-util-key="@ant-design-icons">
.anticon {
  display: inline-block;
  color: inherit;
  font-style: normal;
  line-height: 0;
  text-align: center;
  text-transform: none;
  vertical-align: -0.125em;
  text-rendering: optimizeLegibility;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}

.anticon > * {
  line-height: 1;
}

.anticon svg {
  display: inline-block;
}

.anticon::before {
  display: none;
}

.anticon .anticon-icon {
  display: block;
}

.anticon[tabindex] {
  cursor: pointer;
}

.anticon-spin::before,
.anticon-spin {
  display: inline-block;
  -webkit-animation: loadingCircle 1s infinite linear;
  animation: loadingCircle 1s infinite linear;
}

@-webkit-keyframes loadingCircle {
  100% {
    -webkit-transform: rotate(360deg);
    transform: rotate(360deg);
  }
}

@keyframes loadingCircle {
  100% {
    -webkit-transform: rotate(360deg);
    transform: rotate(360deg);
  }
}
</style><meta chars

Found 0 columns
